In [1]:
import csv
import os
import glob
import time
import random
from multiprocessing import Pool
import tqdm
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
from scipy.interpolate import interp1d
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import pickle as pkl
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.decomposition import PCA,TruncatedSVD,NMF
import dask
from nltk.tag.stanford import StanfordNERTagger
st = StanfordNERTagger('unigrams/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz','unigrams/stanford-ner-2018-02-27/stanford-ner-3.9.1.jar',encoding='utf-8')


In [2]:
br_to_us=pd.read_excel("Book.xlsx")
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))

contextwords_df=pd.read_csv("contexts.csv",sep="\t")
contextwords=contextwords_df.context.tolist()
nouns_df=contextwords_df[contextwords_df.context.str.contains("^.*_n$")]
nouns=nouns_df.context.tolist()


adv_dict=dict(zip(['adv'],['r']))
replacements1={'r1_pos':adv_dict,'r2_pos':adv_dict,'r3_pos':adv_dict,'l1_pos':adv_dict,'l2_pos':adv_dict,'l3_pos':adv_dict}
replacements={'r1':br_to_us_dict,'r2':br_to_us_dict,'r3':br_to_us_dict,'l1':br_to_us_dict,'l2':br_to_us_dict,'l3':br_to_us_dict,'mod':br_to_us_dict,'head':br_to_us_dict,'word':br_to_us_dict}
decades=[2000, 1990, 1980, 1970, 1960, 1950, 1940, 1930, 1920, 1900, 1910,
            1890, 1880, 1870, 1850, 1860, 1840, 1830, 1820, 1810, 1800]
fivegram_list='a_ aa ab ac ad ae af ag ah ai aj ak al am an ao ap aq ar as at au av aw ax ay az b_ ba bb bc bd be bf bg bh bi bj bk bl bm bn bo bp bq br bs bt bu bv bw bx by bz c_ ca cb cc cd ce cf cg ch ci cj ck cl cm cn co cp cq cr cs ct cu cv cw cx cy cz d_ da db dc dd de df dg dh di dj dk dl dm dn do dp dq dr ds dt du dv dw dx dy dz e_ ea eb ec ed ee ef eg eh ei ej ek el em en eo ep eq er es et eu ev ew ex ey ez f_ fa fb fc fd fe ff fg fh fi fj fk fl fm fn fo fp fq fr fs ft fu fv fw fx fy fz g_ ga gb gc gd ge gf gg gh gi gj gk gl gm gn go gp gq gr gs gt gu gv gw gx gy gz h_ ha hb hc hd he hf hg hh hi hj hk hl hm hn ho hp hq hr hs ht hu hv hw hx hy hz i_ ia ib ic id ie if ig ih ii ij ik il im in io ip iq ir is it iu iv iw ix iy iz j_ ja jb jc jd je jf jg jh ji jj jk jl jm jn jo jp jq jr js jt ju jv jw jx jy jz k_ ka kb kc kd ke kf kg kh ki kj kk kl km kn ko kp kq kr ks kt ku kv kw kx ky kz l_ la lb lc ld le lf lg lh li lj lk ll lm ln lo lp lq lr ls lt lu lv lw lx ly lz m_ ma mb mc md me mf mg mh mi mj mk ml mm mn mo mp mq mr ms mt mu mv mw mx my mz n_ na nb nc nd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt nu nv nw nx ny nz o_ oa ob oc od oe of og oh oi oj ok ol om on oo op oq or os ot other ou ov ow ox oy oz p_ pa pb pc pd pe pf pg ph pi pj pk pl pm pn po pp pq pr ps pt pu punctuation pv pw px py pz q_ qa qb qc qd qe qf qg qh qi qj ql qm qn qo qp qq qr qs qt qu qv qw qx qy qz r_ ra rb rc rd re rf rg rh ri rj rk rl rm rn ro rp rq rr rs rt ru rv rw rx ry rz s_ sa sb sc sd se sf sg sh si sj sk sl sm sn so sp sq sr ss st su sv sw sx sy sz t_ ta tb tc td te tf tg th ti tj tk tl tm tn to tp tq tr ts tt tu tv tw tx ty tz u_ ua ub uc ud ue uf ug uh ui uj uk ul um un uo up uq ur us ut uu uv uw ux uy uz v_ va vb vc vd ve vf vg vh vi vj vk vl vm vn vo vp vq vr vs vt vu vv vw vx vy vz w_ wa wb wc wd we wf wg wh wi wj wk wl wm wn wo wp wq wr ws wt wu wv ww wx wy wz x_ xa xb xc xd xe xf xg xh xi xj xk xl xm xn xo xp xq xr xs xt xu xv xw xx xy xz y_ ya yb yc yd ye yf yg yh yi yj yk yl ym yn yo yp yq yr ys yt yu yv yw yx yy yz z_ za zb zc zd ze zf zg zh zi zj zk zl zm zn zo zp zq zr zs zt zu zv zw zx zy zz'

#temp_list="j_ ja jb jc jd je jf jg jh ji jj jk jl jm jn jo jp jq jr js jt ju jv jw jx jy jz"
fivegram_list=fivegram_list.split(" ")
np.random.shuffle(fivegram_list)
for letter in fivegram_list:
    if len(letter)>2:
        #print(letter)
        fivegram_list.remove(letter)
    #elif len(letter.strip("_"))==1:
        #print(letter)
        #bigram_list.remove(letter)
def partition(lst, n):
    division = len(lst) / float(n)
    return [ lst[int(round(division * i)): int(round(division * (i + 1)))] for i in range(n) ]


partitioned_lists=partition((fivegram_list),4)
len(fivegram_list)
def tf(x):
    return(np.log10(x))


def ner_checker(stat):
    temp_list=st.tag(stat)
    if temp_list[0][1]=="PERSON" and temp_list[1][1]=="PERSON":
        return stat,True
    else:
        return stat,False

In [3]:
def tupleconverter(tup):
    dicter={}
    for word,decade in tup:
        #print(word,decade)
        if word in dicter:
            dicter[word].append(decade)
        else:
            dicter[word]=[]
            dicter[word].append(decade)
    return(dicter)


def decremover(dc):
    decade_set=set(list(range(1,19)))
    keep_mods=[]
    unkeep_modes=[]
    for modifier,decades in new_modifiers.items():
        if len(decades)<=2:
            if len(decade_set.intersection(decades))==0:
                unkeep_modes.append(modifier)
    return(unkeep_modes)

def decader(x):
    return(x -x%10)
#fourgrampath="4grams/"
#fourgram_files=glob.glob(os.path.join(fourgrampath, "*.txt"))
import re
def patternmaker(x):
    x=np.array(x.notnull())
    x=x.astype(int)
    #print(x)
    val = ''.join(map(str, x))
    #print(val)
    return val

#modifier_list = pkl.load( open( "modifier_list.p", "rb" ) )
#head_list = pkl.load( open( "head_list.p", "rb" ) )
#words=list(set(modifier_list).union(head_list))
#print(len(words))

def lemma_maker(x, y):
    #print(x,y)
    return lemmatizer.lemmatize(x,y)

In [4]:
def cdsm_word_reducer(df):
    rightgram=reducer(df,align="right",level="word")
    word_right_counts=pd.melt(rightgram.reset_index(),id_vars=['word','decade','count'],value_vars=['r1','r2','r3'])
    
    mid1gram=reducer(df,align="mid1",level="word")
    word_mid1_counts=pd.melt(mid1gram.reset_index(),id_vars=['word','decade','count'],value_vars=['r1','r2','r3','l1'])
    
    mid2gram=reducer(df,align="mid2",level="word")
    word_mid2_counts=pd.melt(mid2gram.reset_index(),id_vars=['word','decade','count'],value_vars=['r1','r2','l2','l1'])
    
    mid3gram=reducer(df,align="mid3",level="word")
    word_mid3_counts=pd.melt(mid3gram.reset_index(),id_vars=['word','decade','count'],value_vars=['l1','l2','l3','r1'])
    
    leftgram=reducer(df,align="left",level="word")
    word_left_counts=pd.melt(leftgram.reset_index(),id_vars=['word','decade','count'],value_vars=['l1','l2','l3'])
    
    words_df=pd.concat([word_right_counts,word_mid1_counts,word_mid2_counts,word_mid3_counts,word_left_counts])
    words_df.dropna(inplace=True)
    words_df=words_df.groupby(['word','value','decade'])['count'].sum().to_frame()
    words_df.reset_index(inplace=True)
    words_df.decade=words_df.decade.astype("int32")
    
    return words_df

def cdsm_phrase_reducer(df):
    rightgram=reducer(df,align="right",level="phrase")
    phrase_right_counts=pd.melt(rightgram.reset_index(),id_vars=['mod','head','decade','count'],value_vars=['r1','r2','r3'])
    
    mid1gram=reducer(df,align="mid1",level="phrase")
    phrase_mid1_counts=pd.melt(mid1gram.reset_index(),id_vars=['mod','head','decade','count'],value_vars=['r1','r2','l1'])
    
    mid2gram=reducer(df,align="mid2",level="phrase")
    phrase_mid2_counts=pd.melt(mid2gram.reset_index(),id_vars=['mod','head','decade','count'],value_vars=['l1','l2','r1'])
    
    leftgram=reducer(df,align="left",level="phrase")
    phrase_left_counts=pd.melt(mid2gram.reset_index(),id_vars=['mod','head','decade','count'],value_vars=['l1','l2','l3'])
    
    phrase_df=pd.concat([phrase_right_counts,phrase_mid1_counts,phrase_mid2_counts,phrase_left_counts])
    phrase_df.dropna(inplace=True)
    phrase_df=phrase_df.groupby(['mod','head','value','decade'])['count'].sum().to_frame()
    phrase_df.reset_index(inplace=True)
    phrase_df.decade=phrase_df.decade.astype("int32")
    
    return phrase_df

In [19]:
def reducer(df,align,oper_type=None,level=None):
    if df.empty==True:
        return df
    if align=="right":
        if level=="word":
            df=df.loc[df.fivegram_pos.str.match("^[a-z]+_noun [a-z]+_.* [a-z-]+_.* [a-z-]+_.* [a-z-]+_.*$")]
            if df.empty==True:
                return df
            df['word_pos'],df['rr1_pos'],df['rr2_pos'],df['rr3_pos'],_=df['fivegram_pos'].str.split(" ").str
        else:
            df=df.loc[df.fivegram_pos.str.match("^[a-z]+_noun [a-z]+_noun [a-z-]+_.* [a-z-]+_.* [a-z-]+_.*$")]
            if level is None:
                df=df.loc[~df.fivegram_pos.str.match("^[a-z]+_noun [a-z]+_noun [a-z-]+_noun [a-z-]+_.* [a-z-]+_.*$")]
            if df.empty==True:
                return df
            df['modifier_pos'],df['head_pos'],df['rr1_pos'],df['rr2_pos'],df['rr3_pos']=df['fivegram_pos'].str.split(" ").str
        #df.replace(replacements,inplace=True)
        df['r3'],df['r3_pos']=df['rr3_pos'].str.split('_', 1).str
        df['r2'],df['r2_pos']=df['rr2_pos'].str.split('_', 1).str
        df['r1'],df['r1_pos']=df['rr1_pos'].str.split('_', 1).str
        df.drop(["rr3_pos","rr2_pos","rr1_pos"],axis=1,inplace=True)
        df.replace(replacements1,inplace=True)
        df['r1_pos']=df['r1_pos'].str[0]
        df['r2_pos']=df['r2_pos'].str[0]
        df['r3_pos']=df['r3_pos'].str[0]

    elif align=="mid1":
        if level=="word":
            df=df.loc[df.fivegram_pos.str.match("^[a-z]+_.* [a-z]+_noun [a-z-]+_.* [a-z-]+_.* [a-z-]+_.*$")]
            if df.empty==True:
                return df
            df['ll1_pos'],df['word_pos'],df['rr1_pos'],df['rr2_pos'],df['rr3_pos']=df['fivegram_pos'].str.split(" ").str
        else:
            df=df.loc[df.fivegram_pos.str.match("^[a-z]+_.* [a-z]+_noun [a-z-]+_noun [a-z-]+_.* [a-z-]+_.*$")]
            if level is None:
                df=df.loc[~df.fivegram_pos.str.match("^[a-z]+_noun [a-z]+_noun [a-z-]+_noun [a-z-]+_noun [a-z-]+_.*$")]
            if df.empty==True:
                return df
            df['ll1_pos'],df['modifier_pos'],df['head_pos'],df['rr1_pos'],df['rr2_pos']=df['fivegram_pos'].str.split(" ").str
        if level=="word":
            df['r3'],df['r3_pos']=df['rr3_pos'].str.split('_', 1).str
            df.drop(["rr3_pos"],axis=1,inplace=True)
        df['r2'],df['r2_pos']=df['rr2_pos'].str.split('_', 1).str
        df['r1'],df['r1_pos']=df['rr1_pos'].str.split('_', 1).str
        df['l1'],df['l1_pos']=df['ll1_pos'].str.split('_', 1).str
        df.drop(["ll1_pos","rr2_pos","rr1_pos"],axis=1,inplace=True)
        df.replace(replacements1,inplace=True)
        df['r1_pos']=df['r1_pos'].str[0]
        df['r2_pos']=df['r2_pos'].str[0]
        df['l1_pos']=df['l1_pos'].str[0]
        if level=="word":
            df['r3_pos']=df['r3_pos'].str[0]

    elif align=="mid2":
        if level=="word":
            df=df.loc[df.fivegram_pos.str.match("^[a-z]+_.* [a-z]+_.* [a-z-]+_noun [a-z-]+_.* [a-z-]+_.*$")]
            if df.empty==True:
                return df
            df['ll1_pos'],df['ll2_pos'],df['word_pos'],df['rr1_pos'],df['rr2_pos']=df['fivegram_pos'].str.split(" ").str
        else:
            
            df=df.loc[df.fivegram_pos.str.match("^[a-z]+_.* [a-z]+_.* [a-z-]+_noun [a-z-]+_noun [a-z-]+_.*$")]
            if level is None:
                df=df.loc[~df.fivegram_pos.str.match("^[a-z]+_.* [a-z]+_noun [a-z-]+_noun [a-z-]+_noun [a-z-]+_noun$")]
            if df.empty==True:
                return df
            df['ll1_pos'],df['ll2_pos'],df['modifier_pos'],df['head_pos'],df['rr1_pos']=df['fivegram_pos'].str.split(" ").str 
        if level=="word":
            df['r2'],df['r2_pos']=df['rr2_pos'].str.split('_', 1).str
            df.drop(["rr2_pos"],axis=1,inplace=True)
        df['r1'],df['r1_pos']=df['rr1_pos'].str.split('_', 1).str
        df['l2'],df['l2_pos']=df['ll2_pos'].str.split('_', 1).str
        df['l1'],df['l1_pos']=df['ll1_pos'].str.split('_', 1).str
        
        df.drop(["rr1_pos","ll2_pos","ll1_pos"],axis=1,inplace=True)
        df.replace(replacements1,inplace=True)
        df['r1_pos']=df['r1_pos'].str[0]
        df['l2_pos']=df['l2_pos'].str[0]
        df['l1_pos']=df['l1_pos'].str[0]
        if level=="word":
            df['r2_pos']=df['r2_pos'].str[0]
    
    elif align=="mid3":
        df=df.loc[df.fivegram_pos.str.match("^[a-z]+_.* [a-z]+_.* [a-z-]+_.* [a-z-]+_noun [a-z-]+_.*$")]
        if df.empty==True:
            return df
        df['ll1_pos'],df['ll2_pos'],df['ll3_pos'],df['word_pos'],df['rr1_pos']=df['fivegram_pos'].str.split(" ").str
        df['l3'],df['l3_pos']=df['ll3_pos'].str.split('_', 1).str
        df['l2'],df['l2_pos']=df['ll2_pos'].str.split('_', 1).str
        df['l1'],df['l1_pos']=df['ll1_pos'].str.split('_', 1).str
        df['r1'],df['r1_pos']=df['rr1_pos'].str.split('_', 1).str
        
        df.drop(["ll3_pos","ll2_pos","ll1_pos","rr1_pos"],axis=1,inplace=True)
        df.replace(replacements1,inplace=True)
        df['l1_pos']=df['l1_pos'].str[0]
        df['l2_pos']=df['l2_pos'].str[0]
        df['l3_pos']=df['l3_pos'].str[0]
        df['r1_pos']=df['r1_pos'].str[0]
        
    elif align=="left":
        
        if level=="word":
            df=df.loc[df.fivegram_pos.str.match("^[a-z]+_.* [a-z]+_.* [a-z-]+_.* [a-z-]+_.* [a-z-]+_noun$")]
            if df.empty==True:
                return df
            _,df['ll1_pos'],df['ll2_pos'],df['ll3_pos'],df['word_pos']=df['fivegram_pos'].str.split(" ").str
        else:
            df=df.loc[df.fivegram_pos.str.match("^[a-z]+_.* [a-z]+_.* [a-z-]+_.* [a-z-]+_noun [a-z-]+_noun$")]
            if level is None:
                df=df.loc[~df.fivegram_pos.str.match("^[a-z]+_.* [a-z]+_.* [a-z-]+_noun [a-z-]+_noun [a-z-]+_noun$")]
            if df.empty==True:
                return df
            df['ll1_pos'],df['ll2_pos'],df['ll3_pos'],df['modifier_pos'],df['head_pos']=df['fivegram_pos'].str.split(" ").str
        df['l3'],df['l3_pos']=df['ll3_pos'].str.split('_', 1).str
        df['l2'],df['l2_pos']=df['ll2_pos'].str.split('_', 1).str
        df['l1'],df['l1_pos']=df['ll1_pos'].str.split('_', 1).str
        
        df.drop(["ll3_pos","ll2_pos","ll1_pos"],axis=1,inplace=True)
        df.replace(replacements1,inplace=True)
        df['l1_pos']=df['l1_pos'].str[0]
        df['l2_pos']=df['l2_pos'].str[0]
        df['l3_pos']=df['l3_pos'].str[0]
        
    if level=="word":
        df['word'],df['w_pos']=df['word_pos'].str.split('_', 1).str
        df['w_pos']="n"
        df.drop(["fivegram_pos","word_pos"],axis=1,inplace=True)
        df.replace(replacements,inplace=True)
        df.dropna(inplace=True)
        df['word']=np.vectorize(lemma_maker)(df['word'], df['w_pos'])
        df['word']=df['word']+"_n"
        df=df.loc[df['word'].isin(nouns)]
    else:
        df['head'],df['h_pos']=df['head_pos'].str.split('_', 1).str
        df['mod'],df['m_pos']=df['modifier_pos'].str.split('_', 1).str
        df['h_pos']="n"
        df['m_pos']="n"
        df.drop(["fivegram_pos","modifier_pos","head_pos"],axis=1,inplace=True)
        df.replace(replacements,inplace=True)
        df.dropna(inplace=True)
        df['head']=np.vectorize(lemma_maker)(df['head'], df['h_pos'])
        df['mod']=np.vectorize(lemma_maker)(df['mod'], df['m_pos'])
        df['head']=df['head']+"_n"
        df['mod']=df['mod']+"_n"
        df=df.loc[df['mod'].isin(nouns) & df['head'].isin(nouns)]
        
    if df.empty==True:
        return df
    
    #print(df)
    if align=="right":
        df['r1']=np.vectorize(lemma_maker)(df['r1'], df['r1_pos'])
        df['r2']=np.vectorize(lemma_maker)(df['r2'], df['r2_pos'])
        df['r3']=np.vectorize(lemma_maker)(df['r3'], df['r3_pos'])
        
        df['r1']=df['r1']+"_"+df['r1_pos']
        df['r2']=df['r2']+"_"+df['r2_pos']
        df['r3']=df['r3']+"_"+df['r3_pos']
        df=df.loc[df['r1'].isin(contextwords) & df['r2'].isin(contextwords) & df['r3'].isin(contextwords)]
        
    elif align=="mid1":
        df['r1']=np.vectorize(lemma_maker)(df['r1'], df['r1_pos'])
        df['r2']=np.vectorize(lemma_maker)(df['r2'], df['r2_pos'])
        df['l1']=np.vectorize(lemma_maker)(df['l1'], df['l1_pos'])
        
        df['r1']=df['r1']+"_"+df['r1_pos']
        df['r2']=df['r2']+"_"+df['r2_pos']
        df['l1']=df['l1']+"_"+df['l1_pos']
        df=df.loc[df['r1'].isin(contextwords) & df['r2'].isin(contextwords) & df['l1'].isin(contextwords)]
        if df.empty==True:
            return df
        if level=="word":
            df['r3']=np.vectorize(lemma_maker)(df['r3'], df['r3_pos'])
            df['r3']=df['r3']+"_"+df['r3_pos']
            df=df.loc[df['r3'].isin(contextwords)]
        
    elif align=="mid2":
        df['l1']=np.vectorize(lemma_maker)(df['l1'], df['l1_pos'])
        df['l2']=np.vectorize(lemma_maker)(df['l2'], df['l2_pos'])
        df['r1']=np.vectorize(lemma_maker)(df['r1'], df['r1_pos'])
        
        df['l1']=df['l1']+"_"+df['l1_pos']
        df['l2']=df['l2']+"_"+df['l2_pos']
        df['r1']=df['r1']+"_"+df['r1_pos']
        df=df.loc[df['r1'].isin(contextwords) & df['l2'].isin(contextwords) & df['l1'].isin(contextwords)]
        if df.empty==True:
            return df
        if level=="word":
            df['r2']=np.vectorize(lemma_maker)(df['r2'], df['r2_pos'])
            df['r2']=df['r2']+"_"+df['r2_pos']
            df=df.loc[df['r2'].isin(contextwords)]
            
    elif align=="mid3":
        df['l1']=np.vectorize(lemma_maker)(df['l1'], df['l1_pos'])
        df['l2']=np.vectorize(lemma_maker)(df['l2'], df['l2_pos'])
        df['l3']=np.vectorize(lemma_maker)(df['l3'], df['l3_pos'])
        df['r1']=np.vectorize(lemma_maker)(df['r1'], df['r1_pos'])
        
        df['l1']=df['l1']+"_"+df['l1_pos']
        df['l2']=df['l2']+"_"+df['l2_pos']
        df['l3']=df['l3']+"_"+df['l3_pos']
        df['r1']=df['r1']+"_"+df['r1_pos']
        df=df.loc[df['r1'].isin(contextwords) & df['l1'].isin(contextwords) & df['l2'].isin(contextwords)& df['l3'].isin(contextwords)]
    
    elif align=="left":
        df['l1']=np.vectorize(lemma_maker)(df['l1'], df['l1_pos'])
        df['l2']=np.vectorize(lemma_maker)(df['l2'], df['l2_pos'])
        df['l3']=np.vectorize(lemma_maker)(df['l3'], df['l3_pos'])

        df['l1']=df['l1']+"_"+df['l1_pos']
        df['l2']=df['l2']+"_"+df['l2_pos']
        df['l3']=df['l3']+"_"+df['l3_pos']
        df=df.loc[df['l3'].isin(contextwords) & df['l2'].isin(contextwords) & df['l1'].isin(contextwords)]

    

        
    if align=="right":
        if level=="word":
            df.groupby(['word','r1','r2','r3','decade'])['count'].sum().to_frame()
        else:    
            df.groupby(['mod','head','r1','r2','r3','decade'])['count'].sum().to_frame()
    elif align=="mid1":
        if level=="word":
            df.groupby(['word','l1','r1','r2','r3','decade'])['count'].sum().to_frame()
        else:
            df.groupby(['mod','head','l1','r1','r2','decade'])['count'].sum().to_frame()
    elif align=="mid2":
        if level=="word":
            df.groupby(['word','l1','l2','r1','r2','decade'])['count'].sum().to_frame()
        else:
            df.groupby(['mod','head','l1','l2','r1','decade'])['count'].sum().to_frame()
    elif align=="left":
        if level=="word":
            df.groupby(['word','l1','l2','l3','decade'])['count'].sum().to_frame()
        else:
            df.groupby(['mod','head','l1','l2','l3','decade'])['count'].sum().to_frame()
    #print(df)   
    return df

In [6]:
def cdsm_reducer(df):

    #Right
    
    rightgram=df
    rightgram=reducer(rightgram,align="right")
    #if rightgram.empty!=True:
    compound_right_counts=pd.melt(rightgram.reset_index(),id_vars=['mod','head','decade','count'],value_vars=['r1','r2','r3'])
    mod_right_counts=pd.melt(rightgram.reset_index(),id_vars=['mod','decade','count'],value_vars=['head','r1','r2'])
    head_right_counts=pd.melt(rightgram.reset_index(),id_vars=['head','decade','count'],value_vars=['mod','r1','r2','r3'])
    
    #Mid1-side
    midgram1=df
    midgram1=reducer(midgram1,align="mid1")
    
    compound_mid1_counts=pd.melt(midgram1.reset_index(),id_vars=['mod','head','decade','count'],value_vars=['l1','r1','r2'])
    mod_mid1_counts=pd.melt(midgram1.reset_index(),id_vars=['mod','decade','count'],value_vars=['head','r1','r2','l1'])
    head_mid1_counts=pd.melt(midgram1.reset_index(),id_vars=['head','decade','count'],value_vars=['mod','l1','r1','r2'])
 

    #Mid2-side
    midgram2=df
    midgram2=reducer(midgram2,align="mid2")
    
    compound_mid2_counts=pd.melt(midgram2.reset_index(),id_vars=['mod','head','decade','count'],value_vars=['l1','l2','r1'])
    mod_mid2_counts=pd.melt(midgram2.reset_index(),id_vars=['mod','decade','count'],value_vars=['head','r1','l2','l1'])
    head_mid2_counts=pd.melt(midgram2.reset_index(),id_vars=['head','decade','count'],value_vars=['mod','l1','l2','r1'])
    
    #Left-side
    leftgram=df
    leftgram=reducer(leftgram,align="left")
    
    compound_left_counts=pd.melt(leftgram.reset_index(),id_vars=['mod','head','decade','count'],value_vars=['l1','l2','l3'])
    mod_left_counts=pd.melt(leftgram.reset_index(),id_vars=['mod','decade','count'],value_vars=['head','l1','l2','l3'])
    head_left_counts=pd.melt(leftgram.reset_index(),id_vars=['head','decade','count'],value_vars=['mod','l1','l2'])
    
    compounds=pd.concat([compound_left_counts,compound_mid1_counts,compound_mid2_counts,compound_right_counts])
    modifiers=pd.concat([mod_left_counts,mod_mid1_counts,mod_mid2_counts,mod_right_counts])
    heads=pd.concat([head_left_counts,head_mid1_counts,head_mid2_counts,head_right_counts])
    #print(modifiers)
    #print(heads)
    
    compounds.dropna(inplace=True)
    compounds=compounds.groupby(['mod','head','value','decade'])['count'].sum().to_frame()
    compounds.reset_index(inplace=True)
    compounds.decade=compounds.decade.astype("int32")
    
    modifiers.dropna(inplace=True)
    modifiers=modifiers.groupby(['mod','value','decade'])['count'].sum().to_frame()
    modifiers.reset_index(inplace=True)
    modifiers.decade=modifiers.decade.astype("int32")
    
    heads.dropna(inplace=True)
    heads=heads.groupby(['head','value','decade'])['count'].sum().to_frame()
    heads.reset_index(inplace=True)
    heads.decade=heads.decade.astype("int32")
    return compounds,modifiers,heads

In [7]:
def dataset_extracter(letter,reduce_type="syntactic"):
    CHUNKSIZE = 1_000_000
    print("Started with letters "+letter)
    cur_time=time.time()
    
    df_list=[]
    path_loc="http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-5gram-20120701-"+letter+".gz"
    dfs   = pd.read_csv(path_loc, compression='gzip', header=None, sep="\t", quotechar='"',usecols=[0,1,2],chunksize=CHUNKSIZE)
    for df in dfs:
        df_list.append(chunked_dataset_extracter(df))
    complete_df=pd.concat(df_list)
    if reduce_type=="semantic":
        compounds,modifiers,heads=cdsm_reducer(complete_df)
    else:
        word_df=cdsm_word_reducer(complete_df)
        phrase_df=cdsm_phrase_reducer(complete_df)
    print("Finished with letters "+letter)
    print("Letters "+letter+" took time "+str(round(time.time()-cur_time,3))+" seconds")
    print("\n")
    if reduce_type=="semantic":
        return compounds,modifiers,heads
    else:
        return word_df,phrase_df

In [8]:
def chunked_dataset_extracter(df):
    df.columns=['fivegram_pos','decade','count']
    df['decade']=np.vectorize(decader)(df['decade'])
    df=df.loc[df.decade.isin(decades)]
    df=df.groupby(['fivegram_pos','decade'])['count'].sum().to_frame()
    df.reset_index(inplace=True)
    df.fivegram_pos=df.fivegram_pos.str.lower()
    df=df.loc[df.fivegram_pos.str.match("^[a-z]+_(noun|verb|adj|adv) [a-z]+_(noun|verb|adj|adv) [a-z-]+_(noun|verb|adj|adv) [a-z-]+_(noun|verb|adj|adv) [a-z-]+_(noun|verb|adj|adv)$")]
    df=df.groupby(['fivegram_pos','decade'])['count'].sum().to_frame()
    df.reset_index(inplace=True)
    df.decade=df.decade.astype("int32")
    return(df)

In [9]:
def semantic_roles():
    num_cores=20
    list_num=1
    curr_num=0
    for partition in partitioned_lists:
        compounds_list=[]
        modifiers_list=[]
        heads_list=[]
        pool = Pool(num_cores)
        print("Started with List "+str(list_num))
        print(partition)
        for temp_compounds,temp_modifiers,temp_heads in pool.imap_unordered(dataset_extracter,partition):
            compounds_list.append(temp_compounds)
            modifiers_list.append(temp_modifiers)    
            heads_list.append(temp_heads)
        
        pool.close()
        pool.join()
        compounds = pd.concat(compounds_list)
        compounds=compounds.groupby(['mod','head','value','decade'])['count'].sum().to_frame()
        compounds.reset_index(inplace=True)
    
        with open('compounds.csv', 'a') as f1:
            compounds.to_csv(f1, header=False,sep="\t")
    
        modifiers=pd.concat(modifiers_list)
        modifiers=modifiers.groupby(['mod','value','decade'])['count'].sum().to_frame()
        modifiers.reset_index(inplace=True)
    
    
        with open('modifiers.csv', 'a') as f1:
            modifiers.to_csv(f1, header=False,sep="\t")
    
        heads=pd.concat(heads_list)
        heads=heads.groupby(['head','value','decade'])['count'].sum().to_frame()
        heads.reset_index(inplace=True)
    
        with open('heads.csv', 'a') as f1:
            heads.to_csv(f1, header=False,sep="\t")
    
        print("Ended with List "+str(list_num))
        curr_num+=len(partition)
        print(str(curr_num)+" of letters processed")
    
        list_num+=1

In [10]:
def syntactic_roles():
    num_cores=20
    list_num=1
    curr_num=0
    for partition in partitioned_lists:
        words_list=[]
        phrases_list=[]
        pool = Pool(num_cores)
        print("Started with List "+str(list_num))
        print(partition)
        for temp_words,temp_phrases in pool.imap_unordered(dataset_extracter,partition):
            words_list.append(temp_words)
            phrases_list.append(temp_phrases)    
        
        pool.close()
        pool.join()
        words = pd.concat(words_list)
        words=words.groupby(['word','value','decade'])['count'].sum().to_frame()
        words.reset_index(inplace=True)
    
        with open('words.csv', 'a') as f1:
            words.to_csv(f1, header=False,sep="\t")
    
        phrases=pd.concat(phrases_list)
        phrases=phrases.groupby(['mod','head','value','decade'])['count'].sum().to_frame()
        phrases.reset_index(inplace=True)
    
        with open('phrases.csv', 'a') as f1:
            phrases.to_csv(f1, header=False,sep="\t")
    
        print("Ended with List "+str(list_num))
        curr_num+=len(partition)
        print(str(curr_num)+" of letters processed")
    
        list_num+=1

In [23]:
syntactic_roles()

Started with letters o_
Started with letters nc
Started with letters hc
Started with letters et
Started with letters wo
Started with letters le
Started with letters tg
Started with letters kp
Started with letters rx
Started with letters yh
Started with letters cx
Started with letters fi
Started with letters ta
Started with letters ig
Started with letters wt
Started with letters aa
Started with letters vn
Started with letters zy
Started with letters cv
Started with letters fe
Started with List 1
['nc', 'kp', 'o_', 'ta', 'et', 'hc', 'le', 'fe', 'vn', 'zy', 'cv', 'aa', 'wo', 'fi', 'rx', 'tg', 'cx', 'yh', 'ig', 'wt', 'oy', 'lw', 'lo', 'pe', 'dq', 'iq', 'ux', 'sn', 'en', 'rg', 'kb', 'mg', 'wa', 'pl', 'tu', 'bj', 'mq', 'os', 'q_', 'bg', 'yb', 'vx', 'my', 'bu', 'bb', 'mn', 'kk', 'ho', 'qh', 'ad', 'fa', 'ds', 'yl', 'h_', 'yw', 'dp', 'w_', 'ij', 'cu', 'ia', 'tz', 'io', 'vm', 'ww', 'ew', 'bl', 'xw', 'rt', 'li', 'ha', 'cf', 'gp', 'tk', 'jb', 'ef', 'e_', 'nt', 'pa', 'iu', 'ee', 'br', 'ji', 'sm', '



Started with letters v_
Finished with letters v_
Letters v_ took time 9.222 seconds


Started with letters uo
Finished with letters uo
Letters uo took time 9.068 seconds


Started with letters xk
Finished with letters xk
Letters xk took time 0.487 seconds


Started with letters yr
Finished with letters yr
Letters yr took time 16.825 seconds


Started with letters nv
Finished with letters le
Letters le took time 1039.553 seconds


Started with letters cc
Finished with letters nv
Letters nv took time 16.538 seconds


Started with letters ax
Finished with letters cc
Letters cc took time 19.679 seconds


Started with letters jl
Finished with letters jl
Letters jl took time 16.608 seconds


Started with letters ma
Finished with letters ax
Letters ax took time 25.143 seconds


Started with letters op
Finished with letters en
Letters en took time 1074.236 seconds


Started with letters ze
Finished with letters ob
Letters ob took time 275.469 seconds


Started with letters hx
Finished with l

Started with letters xq
Started with letters aj
Started with letters jd
Started with letters zd
Started with letters np
Started with letters hn
Started with letters da
Started with letters uy
Started with letters hi
Started with letters vs
Started with letters if
Started with letters lb
Started with letters mf
Started with letters ib
Started with letters wu
Started with letters km
Started with letters mz
Started with letters oi
Started with List 2
['po', 'lb', 'np', 'xq', 'wu', 'ib', 'hn', 'es', 'aj', 'jd', 'da', 'zd', 'if', 'vs', 'hi', 'uy', 'mf', 'km', 'mz', 'oi', 'ex', 'te', 'px', 'od', 'gy', 'gs', 'ga', 'lg', 'pn', 'gh', 'kf', 'fu', 'ug', 'rr', 'bt', 'wq', 'qr', 'ly', 'oe', 'an', 'gf', 'rq', 'kn', 'jp', 'kx', 'ty', 'df', 'za', 'xn', 'ok', 'eb', 'sy', 'ua', 'qx', 'ts', 'fl', 'lm', 'aw', 'rn', 'dx', 'qc', 'xo', 'ez', 'nn', 'xm', 'vg', 'fn', 'uj', 'ht', 'me', 'cz', 'fp', 'ru', 'td', 'tq', 'uz', 'ft', 'jf', 'l_', 'nj', 'el', 'jt', 'ab', 'mh', 'k_', 'or', 'xu', 'pr', 'hl', 'lz', 'ie', '

Letters vv took time 20.446 seconds


Started with letters a_
Finished with letters nx
Letters nx took time 18.053 seconds


Started with letters rv
Finished with letters fl
Letters fl took time 210.301 seconds


Started with letters qu
Finished with letters rv
Letters rv took time 17.293 seconds


Started with letters th
Finished with letters if
Letters if took time 318.648 seconds


Started with letters ky
Finished with letters ky
Letters ky took time 19.204 seconds


Started with letters wm
Finished with letters sy
Letters sy took time 279.124 seconds


Started with letters bs
Finished with letters kn
Letters kn took time 301.062 seconds


Started with letters tj
Finished with letters tj
Letters tj took time 0.413 seconds


Started with letters py
Finished with letters wm
Letters wm took time 17.964 seconds


Started with letters ge
Finished with letters bs
Letters bs took time 18.447 seconds


Started with letters fr
Finished with letters ru
Letters ru took time 233.65 seconds


St

Started with letters zi
Started with letters rl
Started with letters cl
Started with letters pg
Started with letters rf
Started with letters fy
Started with letters uc
Started with letters so
Started with letters cm
Started with letters la
Started with letters jg
Started with letters lc
Started with letters ic
Started with letters pm
Started with letters ii
Started with letters ou
Started with List 3
['au', 'cl', 'bn', 'rw', 'oo', 'fy', 'jg', 'cm', 'zi', 'pg', 'uc', 'ii', 'rl', 'rf', 'so', 'la', 'ic', 'pm', 'lc', 'ou', 'sx', 'zl', 'xa', 'dw', 'ca', 'of', 'ql', 'zq', 'pc', 'wb', 'll', 'vj', 'c_', 'fd', 'jh', 'ju', 'kc', 't_', 'hw', 'zo', 'hh', 'om', 'no', 'mb', 'qo', 'lj', 'ed', 'jx', 'rh', 'pf', 'lt', 'nw', 'qq', 'tx', 'cb', 'xz', 'zh', 'd_', 'ro', 'tr', 'ea', 'rd', 'sc', 'jq', 'x_', 'sh', 'dg', 'bv', 'ip', 'ag', 'yi', 'fs', 'oj', 'yg', 'ye', 'im', 'ks', 'cd', 'qa', 'rj', 'mc', 'sf', 'm_', 'yq', 'to', 'kt', 'vl', 'wp', 'wz', 'vb', 'yt', 'ox', 'gj', 'uk', 'lu', 'pw', 'qm', 'i_', 'uf', '


Started with letters i_
Finished with letters lu
Letters lu took time 75.502 seconds


Started with letters uf
Finished with letters ju
Letters ju took time 364.385 seconds


Started with letters ja
Finished with letters ou
Letters ou took time 412.331 seconds


Started with letters hz
Finished with letters uf
Letters uf took time 18.095 seconds


Started with letters kz
Finished with letters kz
Letters kz took time 0.46 seconds


Started with letters iy
Finished with letters iy
Letters iy took time 0.63 seconds


Started with letters va
Finished with letters hz
Letters hz took time 13.433 seconds


Started with letters mt
Finished with letters mt
Letters mt took time 18.786 seconds


Started with letters xe
Finished with letters cl
Letters cl took time 444.706 seconds


Started with letters lx
Finished with letters lx
Letters lx took time 18.438 seconds


Started with letters bw
Finished with letters xe
Letters xe took time 19.185 seconds


Started with letters zk
Finished with lette

Started with letters sg
Started with letters s_
Started with letters cg
Started with letters zt
Started with letters tb
Started with letters wy
Started with letters nm
Started with letters ng
Started with letters ln
Started with letters si
Started with letters qi
Started with letters r_
Started with letters xt
Started with letters il
Started with letters pu
Started with letters ff
Started with letters al
Started with List 4
['bi', 'ih', 'ae', 'zt', 'ng', 'cg', 'il', 'sg', 'tb', 's_', 'nm', 'wy', 'si', 'ff', 'al', 'pu', 'ln', 'xt', 'qi', 'r_', 'gx', 'gg', 'zw', 'er', 'lv', 'kj', 'vf', 'ya', 'tp', 'zz', 'rb', 'ey', 'qb', 'ry', 'ka', 'sr', 'zr', 'wh', 'uw', 'fw', 'hr', 'vc', 'jk', 'yd', 'eq', 'nu', 'hm', 'oa', 'zv', 'qs', 'yx', 'jn', 'ek', 'st', 'sd', 'ul', 'tv', 'ni', 'pb', 'ir', 'oc', 'af', 'cq', 'pp', 'nq', 'gc', 'lf', 'yy', 'ev', 'cn', 'yc', 'pz', 'ov', 'dh', 'f_', 'ba', 'lq', 'zg', 'db', 've', 'eg', 'bx', 'ep', 'ot', 'rk', 'gz', 'kq', 'hk', 'rp', 'vq', 'fv', 'wj', 'cs', 'zu', 'at', '



Started with letters su
Finished with letters zu
Letters zu took time 20.08 seconds


Started with letters dm
Finished with letters fh
Letters fh took time 17.859 seconds


Started with letters hq
Finished with letters hq
Letters hq took time 2.165 seconds


Started with letters ym
Finished with letters ni
Letters ni took time 142.301 seconds


Started with letters dj
Finished with letters ym
Letters ym took time 0.645 seconds


Started with letters vu
Finished with letters ub
Letters ub took time 19.805 seconds


Started with letters vy
Finished with letters vy
Letters vy took time 6.124 seconds


Started with letters dn
Finished with letters dm
Letters dm took time 17.715 seconds


Started with letters dl
Finished with letters oc
Letters oc took time 137.749 seconds


Started with letters vz
Finished with letters dj
Letters dj took time 17.681 seconds


Started with letters bk
Finished with letters vu
Letters vu took time 22.79 seconds


Started with letters oq
Finished with letter